In [4]:
from statistics import median_grouped

import pandas as pd
import numpy as np
import shap
from pathlib import Path

from biopsykit.classification.model_selection import SklearnPipelinePermuter

from pepbench.io import compute_mae_std_from_metric_summary

In [ ]:
model_path_b_point = Path("../../../results/models/b-point")

### Load B-Point Pipeline Permuters

In [18]:
permuter_list = []
permuter_b_point_without_rr_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath("without-rr-interval/b_point_regression_hpc_0_baseline_result.pkl")).metric_summary()
permuter_b_point_without_rr_ms["Dataset"] = "Without-RR-Interval"
permuter_list.append(permuter_b_point_without_rr_ms)
permuter_b_point_without_rr_include_nan_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath("without-rr-interval/b_point_regression_hpc_0_baseline_result_include_nan.pkl")).metric_summary()
permuter_b_point_without_rr_include_nan_ms["Dataset"] = "Without-RR-Interval-Include-Nan"
permuter_b_point_without_rr_include_nan_ms["pipeline_reduce_dim"] = "None"
permuter_b_point_without_rr_include_nan_ms = permuter_b_point_without_rr_include_nan_ms.reset_index().set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"])
permuter_list.append(permuter_b_point_without_rr_include_nan_ms)
permuter_b_point_without_rr_median_imputed_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath("without-rr-interval/b_point_regression_hpc_0_baseline_result_median_imputed.pkl")).metric_summary()
permuter_b_point_without_rr_median_imputed_ms["Dataset"] = "Without-RR-Interval-Median-Imputed"
permuter_list.append(permuter_b_point_without_rr_median_imputed_ms)
permuter_b_point_rr_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath("rr-interval/b_point_regression_hpc_0_baseline_result_rr_interval.pkl")).metric_summary()
permuter_b_point_rr_ms["Dataset"] = "RR-Interval"
permuter_list.append(permuter_b_point_rr_ms)
permuter_b_point_rr_include_nan_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath("rr-interval/b_point_regression_hpc_0_baseline_result_rr_interval_include_nan.pkl")).metric_summary()
permuter_b_point_rr_include_nan_ms["Dataset"] = "RR-Interval-Include-Nan"
permuter_b_point_rr_include_nan_ms["pipeline_reduce_dim"] = "None"
permuter_b_point_rr_include_nan_ms = permuter_b_point_rr_include_nan_ms.reset_index().set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"])
permuter_list.append(permuter_b_point_rr_include_nan_ms)
permuter_b_point_rr_median_imputed_ms = SklearnPipelinePermuter.from_pickle(model_path_b_point.joinpath("rr-interval/b_point_regression_hpc_0_baseline_result_rr_interval_median_imputed.pkl")).metric_summary()
permuter_b_point_rr_median_imputed_ms["Dataset"] = "RR-Interval-Median-Imputed"
permuter_list.append(permuter_b_point_rr_median_imputed_ms)

In [46]:
merged_permuter = permuter_b_point_without_rr_ms
for p in permuter_list[1:]:
    merged_permuter = pd.concat([merged_permuter, p])
merged_permuter = merged_permuter.reset_index()
merged_permuter = merged_permuter.set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf", "Dataset"])
merged_permuter

conf_matrix  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                  
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval              [0.0]   
                                    RandomForestRegressor Without-RR-Interval              [0.0]   
                                    SVR                   Without-RR-Interval              [0.0]   
                                    KNeighborsRegressor   Without-RR-Interval              [0.0]   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval              [0.0]   
...                                                                                          ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed       [0.0]   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed       [0.0]   
                                    RandomForestRegressor RR-Interval-Median-Imputed       [0.0]   
                                    SVR                   RR-Interval-Median-Imputed       [0.0]   
                                    KNeighborsRegressor   RR-Interval-Median-Imputed       [0.0]   

                                                                                     conf_matrix_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                        
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval                       []   
                                    RandomForestRegressor Without-RR-Interval                       []   
                                    SVR                   Without-RR-Interval                       []   
                                    KNeighborsRegressor   Without-RR-Interval                       []   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval                       []   
...                                                                                                ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed                []   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed                []   
                                    RandomForestRegressor RR-Interval-Median-Imputed                []   
                                    SVR                   RR-Interval-Median-Imputed                []   
                                    KNeighborsRegressor   RR-Interval-Median-Imputed                []   

                                                                                                                            true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                                                         
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    RandomForestRegressor Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    SVR                   Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    KNeighborsRegressor   Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
...                                                                                                                                 ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
               

In [13]:
algo_name_mapping = {
    "StandardScaler": "SS",
    "MinMaxScaler": "MinMax",
    "SelectFromModel": "SFM",
    "SelectKBest": "KBest",
    "DecisionTreeRegressor": "DTR",
    "RandomForestRegressor": "RFR",
    "KNeighborsRegressor": "KNR",
}

In [58]:
merged_permuter_renamed = merged_permuter.copy()
merged_permuter_renamed = merged_permuter_renamed.rename(algo_name_mapping)
merged_permuter_renamed

conf_matrix  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                  
SS              SFM                 DTR          Without-RR-Interval              [0.0]   
                                    RFR          Without-RR-Interval              [0.0]   
                                    SVR          Without-RR-Interval              [0.0]   
                                    KNR          Without-RR-Interval              [0.0]   
                KBest               DTR          Without-RR-Interval              [0.0]   
...                                                                                 ...   
MinMax          SFM                 KNR          RR-Interval-Median-Imputed       [0.0]   
                KBest               DTR          RR-Interval-Median-Imputed       [0.0]   
                                    RFR          RR-Interval-Median-Imputed       [0.0]   
                                    SVR          RR-Interval-Median-Imputed       [0.0]   
                                    KNR          RR-Interval-Median-Imputed       [0.0]   

                                                                            conf_matrix_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                        
SS              SFM                 DTR          Without-RR-Interval                       []   
                                    RFR          Without-RR-Interval                       []   
                                    SVR          Without-RR-Interval                       []   
                                    KNR          Without-RR-Interval                       []   
                KBest               DTR          Without-RR-Interval                       []   
...                                                                                       ...   
MinMax          SFM                 KNR          RR-Interval-Median-Imputed                []   
                KBest               DTR          RR-Interval-Median-Imputed                []   
                                    RFR          RR-Interval-Median-Imputed                []   
                                    SVR          RR-Interval-Median-Imputed                []   
                                    KNR          RR-Interval-Median-Imputed                []   

                                                                                                                   true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                                                         
SS              SFM                 DTR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    RFR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    SVR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                                    KNR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                KBest               DTR          Without-RR-Interval         [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
...                                                                                                                        ...   
MinMax          SFM                 KNR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                KBest               DTR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                                    RFR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                                    SVR          RR-Interval-Median-Imputed  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                                    KNR          RR-Interval-Median-Im

In [42]:
merged_permuter_renamed["EstimatorID"] = [f"{scaler}_{reduce_dim}_{reg}_{data}" for scaler, reduce_dim, reg, data in merged_permuter_renamed.index]

In [43]:
# Reset the index to be able to save the dataframe in the .json format
merged_permuter_renamed = merged_permuter_renamed.reset_index()
merged_permuter_renamed

,pipeline_scaler,pipeline_reduce_dim,pipeline_clf,Dataset,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,...,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4,EstimatorID
0,SS,SFM,DTR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[368.0, 342.0, 376.0, 352.5, 336.0, 319.0, 334...","[[368.0, 342.0, 376.0, 352.5, 336.0, 319.0, 33...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.423520,0.774451,9.380511,7.691833,8.839198,7.338353,8.867702,SS_SFM_DTR_Without-RR-Interval
1,SS,SFM,RFR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[358.1211657251767, 341.0184249193839, 373.052...","[[358.1211657251767, 341.0184249193839, 373.05...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.102284,0.684734,8.747933,7.302502,8.708227,7.236630,8.516127,SS_SFM_RFR_Without-RR-Interval
2,SS,SFM,SVR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[370.5407480486517, 350.8508283665991, 373.453...","[[370.5407480486517, 350.8508283665991, 373.45...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",7.769715,0.648729,8.365773,6.895291,8.444819,7.097069,8.045623,SS_SFM_SVR_Without-RR-Interval
3,SS,SFM,KNR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[362.6237180099921, 351.03243128230656, 366.35...","[[362.6237180099921, 351.03243128230656, 366.3...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.651584,0.745946,9.409994,7.782687,9.069985,7.713554,9.281699,SS_SFM_KNR_Without-RR-Interval
4,SS,KBest,DTR,Without-RR-Interval,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[365.0, 338.0, 378.0, 368.0, 340.0, 312.0, 338...","[[365.0, 338.0, 378.0, 368.0, 340.0, 312.0, 33...",...,"[2257, 2258, 2259, 2260, 2261, 2262, 2263, 226...","[[2257, 2258, 2259, 2260, 2261, 2262, 2263, 22...",8.172423,0.630880,8.918337,7.502849,8.917644,7.546588,7.976695,SS_KBest_DTR_Without-RR-Interval
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,MinMax,SFM,KNR,RR-Interval-Median-Imputed,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[521.465974410588, 518.2383779299331, 486.1810...","[[521.465974410588, 518.2383779299331, 486.181...",...,"[318, 319, 320, 321, 322, 323, 324, 325, 326, ...","[[318, 319, 320, 321, 322, 323, 324, 325, 326,...",9.467910,0.942626,8.606630,9.455270,10.279094,8.266239,10.732316,MinMax_SFM_KNR_RR-Interval-Median-Imputed
68,MinMax,KBest,DTR,RR-Interval-Median-Imputed,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[520.0, 518.0, 497.0, 494.0, 500.0, 339.0, 514...","[[520.0, 518.0, 497.0, 494.0, 500.0, 339.0, 51...",...,"[318, 319, 320, 321, 322, 323, 324, 325, 326, ...","[[318, 319, 320, 321, 322, 323, 324, 325, 326,...",8.970884,1.230736,7.936165,9.061731,9.901014,7.300045,10.655466,MinMax_KBest_DTR_RR-Interval-Median-Imputed
69,MinMax,KBest,RFR,RR-Interval-Median-Imputed,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[524.9458922446663, 519.5830944121237, 491.552...","[[524.9458922446663, 519.

In [44]:
merged_permuter_renamed.to_json(model_path_b_point.joinpath("permuter_summary/merged_b_point_permuter.json"), orient="records", lines=True)

### Load Q-Peak Pipeline Permuters

In [5]:
model_path_q_peak = Path("../../../results/models/q-peak")

In [7]:
permuter_list_q_peak = []
permuter_q_peak_without_rr_ms = SklearnPipelinePermuter.from_pickle(model_path_q_peak.joinpath("without-rr-interval/q_peak_regression_hpc_0_baseline_result.pkl")).metric_summary()
permuter_q_peak_without_rr_ms["Dataset"] = "Without-RR-Interval"
permuter_list_q_peak.append(permuter_q_peak_without_rr_ms)
permuter_q_peak_without_rr_include_nan_ms = SklearnPipelinePermuter.from_pickle(model_path_q_peak.joinpath("without-rr-interval/q_peak_regression_hpc_0_baseline_result_include_nan.pkl")).metric_summary()
permuter_q_peak_without_rr_include_nan_ms["Dataset"] = "Without-RR-Interval-Include-Nan"
permuter_q_peak_without_rr_include_nan_ms["pipeline_reduce_dim"] = "None"
permuter_q_peak_without_rr_include_nan_ms = permuter_q_peak_without_rr_include_nan_ms.reset_index().set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"])
permuter_list_q_peak.append(permuter_q_peak_without_rr_include_nan_ms)
permuter_q_peak_without_rr_median_imputed_ms = SklearnPipelinePermuter.from_pickle(model_path_q_peak.joinpath("without-rr-interval/q_peak_regression_hpc_0_baseline_result_median_imputed.pkl")).metric_summary()
permuter_q_peak_without_rr_median_imputed_ms["Dataset"] = "Without-RR-Interval-Median-Imputed"
permuter_list_q_peak.append(permuter_q_peak_without_rr_median_imputed_ms)
permuter_q_peak_rr_ms = SklearnPipelinePermuter.from_pickle(model_path_q_peak.joinpath("rr-interval/q_peak_regression_hpc_0_baseline_result_rr_interval.pkl")).metric_summary()
permuter_q_peak_rr_ms["Dataset"] = "RR-Interval"
permuter_list_q_peak.append(permuter_q_peak_rr_ms)
permuter_q_peak_rr_include_nan_ms = SklearnPipelinePermuter.from_pickle(model_path_q_peak.joinpath("rr-interval/q_peak_regression_hpc_0_baseline_result_rr_interval_include_nan.pkl")).metric_summary()
permuter_q_peak_rr_include_nan_ms["Dataset"] = "RR-Interval-Include-Nan"
permuter_q_peak_rr_include_nan_ms["pipeline_reduce_dim"] = "None"
permuter_q_peak_rr_include_nan_ms = permuter_q_peak_rr_include_nan_ms.reset_index().set_index(["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"])
permuter_list_q_peak.append(permuter_q_peak_rr_include_nan_ms)
permuter_q_peak_rr_median_imputed_ms = SklearnPipelinePermuter.from_pickle(model_path_q_peak.joinpath("rr-interval/q_peak_regression_hpc_0_baseline_result_rr_interval_median_imputed.pkl")).metric_summary()
permuter_q_peak_rr_median_imputed_ms["Dataset"] = "RR-Interval-Median-Imputed"
permuter_list_q_peak.append(permuter_q_peak_rr_median_imputed_ms)

In [8]:
merged_permuter_q_peak = permuter_q_peak_without_rr_ms
for p in permuter_list_q_peak[1:]:
    merged_permuter_q_peak = pd.concat([merged_permuter_q_peak, p])
merged_permuter_q_peak = merged_permuter_q_peak.reset_index()
merged_permuter_q_peak = merged_permuter_q_peak.set_index(['pipeline_scaler', 'pipeline_reduce_dim', 'pipeline_clf', 'Dataset'])
merged_permuter_q_peak

conf_matrix  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                  
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval              [0.0]   
                                    RandomForestRegressor Without-RR-Interval              [0.0]   
                                    SVR                   Without-RR-Interval              [0.0]   
                                    KNeighborsRegressor   Without-RR-Interval              [0.0]   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval              [0.0]   
...                                                                                          ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed       [0.0]   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed       [0.0]   
                                    RandomForestRegressor RR-Interval-Median-Imputed       [0.0]   
                                    SVR                   RR-Interval-Median-Imputed       [0.0]   
                                    KNeighborsRegressor   RR-Interval-Median-Imputed       [0.0]   

                                                                                     conf_matrix_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                        
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval                       []   
                                    RandomForestRegressor Without-RR-Interval                       []   
                                    SVR                   Without-RR-Interval                       []   
                                    KNeighborsRegressor   Without-RR-Interval                       []   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval                       []   
...                                                                                                ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed                []   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed                []   
                                    RandomForestRegressor RR-Interval-Median-Imputed                []   
                                    SVR                   RR-Interval-Median-Imputed                []   
                                    KNeighborsRegressor   RR-Interval-Median-Imputed                []   

                                                                                                                            true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf          Dataset                                                                         
StandardScaler  SelectFromModel     DecisionTreeRegressor Without-RR-Interval         [260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...   
                                    RandomForestRegressor Without-RR-Interval         [260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...   
                                    SVR                   Without-RR-Interval         [260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...   
                                    KNeighborsRegressor   Without-RR-Interval         [260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...   
                SelectKBest         DecisionTreeRegressor Without-RR-Interval         [260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...   
...                                                                                                                                 ...   
MinMaxScaler    SelectFromModel     KNeighborsRegressor   RR-Interval-Median-Imputed  [302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...   
                SelectKBest         DecisionTreeRegressor RR-Interval-Median-Imputed  [302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...   
               

In [11]:
merged_permuter_q_peak["mean_test_neg_mean_absolute_error"].sort_values(ascending=True)

pipeline_scaler  pipeline_reduce_dim  pipeline_clf           Dataset                           
StandardScaler   SelectKBest          SVR                    RR-Interval-Median-Imputed            2.429628
MinMaxScaler     None                 RandomForestRegressor  RR-Interval-Include-Nan               2.466470
StandardScaler   SelectKBest          RandomForestRegressor  RR-Interval-Median-Imputed            2.474768
MinMaxScaler     SelectKBest          DecisionTreeRegressor  RR-Interval-Median-Imputed            2.475040
StandardScaler   None                 RandomForestRegressor  RR-Interval-Include-Nan               2.478083
                                                                                                     ...   
                 SelectFromModel      RandomForestRegressor  Without-RR-Interval-Median-Imputed    3.577456
                                      KNeighborsRegressor    RR-Interval-Median-Imputed            3.658553
MinMaxScaler     SelectFromModel      KN

In [72]:
merged_permuter_q_peak.index.get_level_values('Dataset').value_counts()

Dataset
Without-RR-Interval                   16
Without-RR-Interval-Median-Imputed    16
RR-Interval-Median-Imputed            16
RR-Interval                           16
Without-RR-Interval-Include-Nan        4
RR-Interval-Include-Nan                4
Name: count, dtype: int64

In [14]:
merged_permuter_q_peak_renamed = merged_permuter_q_peak.copy()
merged_permuter_q_peak_renamed = merged_permuter_q_peak_renamed.rename(algo_name_mapping)
merged_permuter_q_peak_renamed[merged_permuter_q_peak_renamed.index.get_level_values('Dataset') == "RR-Interval-Median-Imputed"]

conf_matrix  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                  
SS              SFM                 DTR          RR-Interval-Median-Imputed       [0.0]   
                                    RFR          RR-Interval-Median-Imputed       [0.0]   
                                    SVR          RR-Interval-Median-Imputed       [0.0]   
                                    KNR          RR-Interval-Median-Imputed       [0.0]   
                KBest               DTR          RR-Interval-Median-Imputed       [0.0]   
                                    RFR          RR-Interval-Median-Imputed       [0.0]   
                                    SVR          RR-Interval-Median-Imputed       [0.0]   
                                    KNR          RR-Interval-Median-Imputed       [0.0]   
MinMax          SFM                 DTR          RR-Interval-Median-Imputed       [0.0]   
                                    RFR          RR-Interval-Median-Imputed       [0.0]   
                                    SVR          RR-Interval-Median-Imputed       [0.0]   
                                    KNR          RR-Interval-Median-Imputed       [0.0]   
                KBest               DTR          RR-Interval-Median-Imputed       [0.0]   
                                    RFR          RR-Interval-Median-Imputed       [0.0]   
                                    SVR          RR-Interval-Median-Imputed       [0.0]   
                                    KNR          RR-Interval-Median-Imputed       [0.0]   

                                                                            conf_matrix_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                        
SS              SFM                 DTR          RR-Interval-Median-Imputed                []   
                                    RFR          RR-Interval-Median-Imputed                []   
                                    SVR          RR-Interval-Median-Imputed                []   
                                    KNR          RR-Interval-Median-Imputed                []   
                KBest               DTR          RR-Interval-Median-Imputed                []   
                                    RFR          RR-Interval-Median-Imputed                []   
                                    SVR          RR-Interval-Median-Imputed                []   
                                    KNR          RR-Interval-Median-Imputed                []   
MinMax          SFM                 DTR          RR-Interval-Median-Imputed                []   
                                    RFR          RR-Interval-Median-Imputed                []   
                                    SVR          RR-Interval-Median-Imputed                []   
                                    KNR          RR-Interval-Median-Imputed                []   
                KBest               DTR          RR-Interval-Median-Imputed                []   
                                    RFR          RR-Interval-Median-Imputed                []   
                                    SVR          RR-Interval-Median-Imputed                []   
                                    KNR          RR-Interval-Median-Imputed                []   

                                                                                                                   true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf Dataset                                                                         
SS              SFM                 DTR          RR-Interval-Median-Imputed  [302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...   
                                    RFR          RR-Interval-Median-Imputed  [302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...   
                                    SVR          RR-Interval-Median-Imputed  [302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...   
                                    KNR 

In [15]:
merged_permuter_q_peak_renamed["EstimatorID"] = [f"{scaler}_{reduce_dim}_{reg}_{data}" for scaler, reduce_dim, reg, data in merged_permuter_q_peak_renamed.index]

In [16]:
# Reset the index to be able to save the dataframe in the .json format
merged_permuter_q_peak_renamed = merged_permuter_q_peak_renamed.reset_index()
merged_permuter_q_peak_renamed

,pipeline_scaler,pipeline_reduce_dim,pipeline_clf,Dataset,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,...,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4,EstimatorID
0,SS,SFM,DTR,Without-RR-Interval,[0.0],[],"[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...","[[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 22...","[262.0, 268.0, 224.0, 222.0, 230.0, 238.0, 228...","[[262.0, 268.0, 224.0, 222.0, 230.0, 238.0, 22...",...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",3.427150,0.550110,4.103140,3.929978,3.357143,3.175791,2.569697,SS_SFM_DTR_Without-RR-Interval
1,SS,SFM,RFR,Without-RR-Interval,[0.0],[],"[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...","[[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 22...","[263.4929207791859, 265.38731495354426, 224.69...","[[263.4929207791859, 265.38731495354426, 224.6...",...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",3.414295,0.400475,3.337869,4.048284,3.442569,3.454331,2.788422,SS_SFM_RFR_Without-RR-Interval
2,SS,SFM,SVR,Without-RR-Interval,[0.0],[],"[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...","[[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 22...","[263.14518805506714, 264.3559850875987, 225.57...","[[263.14518805506714, 264.3559850875987, 225.5...",...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",2.992304,0.466672,2.804118,3.825636,2.896105,3.033372,2.402290,SS_SFM_SVR_Without-RR-Interval
3,SS,SFM,KNR,Without-RR-Interval,[0.0],[],"[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...","[[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 22...","[264.0, 265.6666666666667, 223.83333333333334,...","[[264.0, 265.6666666666667, 223.83333333333334...",...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",3.474861,0.346437,3.297936,3.997204,3.500409,3.625897,2.952862,SS_SFM_KNR_Without-RR-Interval
4,SS,KBest,DTR,Without-RR-Interval,[0.0],[],"[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 226...","[[260.0, 260.0, 222.0, 220.0, 228.0, 236.0, 22...","[262.0, 272.0, 224.0, 223.0, 230.0, 238.0, 227...","[[262.0, 272.0, 224.0, 223.0, 230.0, 238.0, 22...",...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",2.756803,0.339900,3.430004,2.669147,2.592778,2.568778,2.523310,SS_KBest_DTR_Without-RR-Interval
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,MinMax,SFM,KNR,RR-Interval-Median-Imputed,[0.0],[],"[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...","[[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 29...","[295.2142857142857, 295.2142857142857, 308.285...","[[295.2142857142857, 295.2142857142857, 308.28...",...,"[3180, 3181, 3182, 3183, 3184, 3185, 3186, 318...","[[3180, 3181, 3182, 3183, 3184, 3185, 3186, 31...",3.664012,0.296394,3.487422,4.146590,3.668467,3.760756,3.256825,MinMax_SFM_KNR_RR-Interval-Median-Imputed
68,MinMax,KBest,DTR,RR-Interval-Median-Imputed,[0.0],[],"[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...","[[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 29...","[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...","[[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 29...",...,"[3180, 3181, 3182, 3183, 3184, 3185, 3186, 318...","[[3180, 3181, 3182, 3183, 3184, 3185, 3186, 31...",2.475040,0.394347,2.231264,3.104686,2.413041,2.677371,1.948839,MinMax_KBest_DTR_RR-Interval-Median-Imputed
69,MinMax,KBest,RFR,RR-Interval-Median-Imputed,[0.0],[],"[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...","[[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 29...","[301.3785770231559, 299.8084907898399, 307.443...","[[301.3785770231559, 299.808

In [17]:
merged_permuter_q_peak_renamed.to_json(model_path_q_peak.joinpath("permuter_summary/merged_q_peak_permuter.json"), orient="records", lines=True)